In [ ]:
!pip install Flask pyngrok flask-cors
!ngrok config add-authtoken 본인의ngrok인증토큰

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
from pyngrok import ngrok
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import pandas as pd

In [ ]:
# Saju Naming 모델 불러오기
model = AutoModelForCausalLM.from_pretrained("5KLetsGo/saju-naming", device_map='auto', torch_dtype='auto')
tokenizer = AutoTokenizer.from_pretrained("rtzr/ko-gemma-2-9b-it")

pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    tokenizer=tokenizer,
    device_map='auto',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
app = Flask(__name__)
app.config['CORS_HEADERS'] = 'Content-Type'

# Saju Naming 모델 사용
def use_model(prompt):
    pipeline.model.eval()

    message = [{"role": "user", "content": f"{prompt}"}]
    print(prompt)
    input_prompt = pipeline.tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_special_tokens=True
    )

    terminator = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    outputs = pipeline(
        input_prompt,
        max_new_tokens=2048,
        eos_token_id=terminator,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    return outputs[0]["generated_text"][len(input_prompt):]


# 음양력 데이터 불러오기
df = pd.read_csv('./dataset/crawled/yinyang_calendar.csv')

# 날짜 변환하기
def generate_prompt(date_str, gender):
    # 'date' 열에서 입력한 날짜와 일치하는 행을 찾습니다
    if date_str in df['date'].values:
        row = df[df['date'] == date_str].iloc[0]
        date = [row['yeonju'], row['wolju'], row['ilju']]
        wood_num = row['wood']
        fire_num = row['fire']
        water_num = row['water']
        earth_num = row['earth']
        metal_num = row['metal']
        result = f"사주에 따른 자원오행이 목은 {wood_num}개, 화는 {fire_num}개, 수는 {water_num}개, 토는 {earth_num}개, 금은 {metal_num}개일 때, 이 사주를 보완하는 한국 {gender} 이름 짓고 뜻 풀이해줘"
        return result, date
    else:
        return None

In [ ]:
@app.route("/check", methods=["GET"])
@cross_origin()
def check_load_modal():
    return jsonify({"isReady": pipeline is not None})


@app.route("/name", methods=["POST"])
@cross_origin()
def name_suggest():
    mean_five_elements_explain = ''
    prompt_element = ''
    choose_name = []
    choose_name_without_elements = []
    data = request.get_json()
    if len(data['month']) == 1:
        data['month'] = f'0{data["month"]}'
    prompt, saju_date = generate_prompt(f"{data['year']}-{data['month']}-{data['day']}", data['gender'])
    prompt_element = prompt.split("일 때,")[0] + "입니다."
    if prompt is not None:
        output = use_model(prompt)
        output = output.split("\n")

        for i in output[0].split(".")[:-3]:
            mean_five_elements_explain += i
            mean_five_elements_explain += ". "
        choose_name.append(output[3] + "\n" + output[4] + "\n" + output[5])
        choose_name.append(output[7] + "\n" + output[8] + "\n" + output[9])
        choose_name.append(output[11] + "\n" + output[12] + "\n" + output[13])
        choose_name_without_elements.append(output[3] + "\n" + output[4][:-8] + "\n" + output[5][:-8])
        choose_name_without_elements.append(output[7] + "\n" + output[8][:-8] + "\n" + output[9][:-8])
        choose_name_without_elements.append(output[11] + "\n" + output[12][:-8] + "\n" + output[13][:-8])
    else:
        output = f"{data['year']}-{data['month']}-{data['day']}에 해당하는 데이터가 없습니다."

    response = jsonify({
        "prompt": prompt_element,
        "explain": mean_five_elements_explain,
        "year": saju_date[0],
        "month": saju_date[1],
        "day": saju_date[2],
        "name1": choose_name[0],
        "name2": choose_name[1],
        "name3": choose_name[2],
        "wname1": choose_name_without_elements[0],
        "wname2": choose_name_without_elements[1],
        "wname3": choose_name_without_elements[2],
    })
    return response

# 서버가 클라이언트에게 간 응답 뭔지 저장해놓고 클라가 인덱스 보내면 해당 저장값으로 모델에 매개변수 넘김
@app.route("/mean", methods=["POST"])
@cross_origin()
def meaning_generator():
    data = request.get_json()
    name_meaning = use_model(f"{data['name']}\n이 이름의 의미를 풀이해")
    return jsonify({
        "mean": name_meaning[:-1]
    })

In [ ]:
public_url = ngrok.connect(5050)
print(f"Public URL: {public_url}")

app.run(host='0.0.0.0', port=5050)

Public URL: NgrokTunnel: "https://40d8-35-188-5-40.ngrok-free.app" -> "http://localhost:5050"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5050
 * Running on http://172.28.0.12:5050
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 15:07:12] "OPTIONS /check HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 15:07:12] "GET /check HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 15:07:18] "OPTIONS /name HTTP/1.1" 200 -


사주에 따른 자원오행이 목은 1개, 화는 1개, 수는 2개, 토는 0개, 금은 2개일 때, 이 사주를 보완하는 한국 남자 이름 짓고 뜻 풀이해줘


INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 15:07:40] "POST /name HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 15:07:47] "OPTIONS /mean HTTP/1.1" 200 -


동형(堣娿)
동(堣): 모퉁이, 자원오행 토
형(娿): 아리따울, 자원오행 토
이 이름의 의미를 풀이해


INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 15:07:49] "POST /mean HTTP/1.1" 200 -
